In [3]:
# --- Outlook -> descargar adjuntos, unzip y juntar Excels ---
# Requiere: pip install pywin32
import pythoncom, win32com.client as win32
from pathlib import Path
from datetime import datetime, timedelta
import re, os, zipfile, shutil

# ===== CONFIG =====
SAVE_ROOT = Path(r"C:\MedicionesInbox")   # cambia si querés otra ruta
DAYS_BACK = 14                            # ventana de búsqueda
ASUNTO_CLAVE = "MEDICIONES"               # lo de tu captura: "MEDICIONES 09-09-2025"
SUBCARPETA_OUTLOOK = None                 # ej: r"Inbox\Mediciones" o None para Inbox

DIR_ZIPS    = SAVE_ROOT / "zips"
DIR_EXTR    = SAVE_ROOT / "extraidos"
DIR_EXCELS  = SAVE_ROOT / "excels"
for d in [SAVE_ROOT, SAVE_ROOT/"adjuntos", DIR_ZIPS, DIR_EXTR, DIR_EXCELS]:
    d.mkdir(parents=True, exist_ok=True)

def clean(s): return re.sub(r'[\\/:*?"<>|]+', "_", (s or "").strip())[:100] or "sin_asunto"

def safe_unzip(zip_path: Path, dest_dir: Path):
    with zipfile.ZipFile(zip_path) as z:
        for m in z.infolist():
            out = (dest_dir / m.filename).resolve()
            if not str(out).startswith(str(dest_dir.resolve())):  # evita zip-slip
                continue
            if m.is_dir():
                out.mkdir(parents=True, exist_ok=True)
            else:
                out.parent.mkdir(parents=True, exist_ok=True)
                with z.open(m) as src, open(out, "wb") as f: f.write(src.read())

def save_att(att, dst_dir: Path) -> Path:
    dst_dir.mkdir(parents=True, exist_ok=True)
    name = clean(att.FileName); p = dst_dir / name; i = 1
    while p.exists():
        p = dst_dir / f"{Path(name).stem} ({i}){Path(name).suffix}"; i += 1
    att.SaveAsFile(str(p))
    return p

# ===== OUTLOOK =====
pythoncom.CoInitialize()
ns = win32.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = ns.GetDefaultFolder(6)  # 6 = Inbox
if SUBCARPETA_OUTLOOK:
    for part in SUBCARPETA_OUTLOOK.split("\\"): inbox = inbox.Folders[part]

items = inbox.Items
items.Sort("[ReceivedTime]", True)
desde = datetime.now() - timedelta(days=DAYS_BACK)
items = items.Restrict(f"[ReceivedTime] >= '{desde.strftime('%m/%d/%Y %I:%M %p')}'")

n_mail=n_att=n_zip=n_xls=0; excel_paths=[]
for it in items:
    try:
        if it.Class != 43: continue  # olMail
        if ASUNTO_CLAVE not in (it.Subject or "").upper(): continue
        n_mail += 1
        ymd = it.ReceivedTime.strftime("%Y-%m-%d")
        carpeta = f"{ymd}__{clean(it.SenderName)}__{clean(it.Subject)}"
        base = SAVE_ROOT/"adjuntos"/carpeta; base.mkdir(parents=True, exist_ok=True)

        for att in list(it.Attachments):
            p = save_att(att, base); n_att += 1
            suf = p.suffix.lower()
            if suf == ".zip":
                n_zip += 1
                shutil.copy2(p, (DIR_ZIPS/carpeta).mkdir(parents=True, exist_ok=True) or (DIR_ZIPS/carpeta/p.name))
                outdir = DIR_EXTR/carpeta/p.stem; safe_unzip(p, outdir)
                for r,_,fs in os.walk(outdir):
                    for f in fs:
                        if f.lower().endswith((".xlsx",".xls",".xlsm")):
                            src = Path(r)/f; trg = DIR_EXCELS/carpeta; trg.mkdir(parents=True, exist_ok=True)
                            dst = trg/src.name; shutil.copy2(src, dst); excel_paths.append(dst); n_xls += 1
            elif suf in (".xlsx",".xls",".xlsm"):
                trg = DIR_EXCELS/carpeta; trg.mkdir(parents=True, exist_ok=True)
                dst = trg/p.name; shutil.copy2(p, dst); excel_paths.append(dst); n_xls += 1
    except Exception as e:
        print("Error en un correo:", e)

print(f"Correos procesados: {n_mail}")
print(f"Adjuntos guardados: {n_att}")
print(f"ZIPs extraídos:     {n_zip}")
print(f"Excels detectados:  {n_xls}")
for p in excel_paths[:10]: print(" -", p)
print("Base:", SAVE_ROOT.resolve())


Correos procesados: 346
Adjuntos guardados: 2698
ZIPs extraídos:     888
Excels detectados:  406
 - C:\MedicionesInbox\excels\2025-09-12__AREVALO, GASTON MATIAS__MEDICIONES 12-09-2025 - LP-LM\Parte Diario Ecodinamometria M-1544 - 12-09-2025.xlsx
 - C:\MedicionesInbox\excels\2025-09-12__AREVALO, GASTON MATIAS__MEDICIONES 12-09-2025 - LP-LM\Parte Diario Ecodinamometría M-1545 - 12-09-2025.xls
 - C:\MedicionesInbox\excels\2025-09-09__VARGAS, SERGIO ALEXANDER__MEDICIONES 09-09-2025\Parte Diario Ecodinamometría M-1543 09-09-2025.xlsx
 - C:\MedicionesInbox\excels\2025-09-09__VARGAS, SERGIO ALEXANDER__MEDICIONES 09-09-2025\Parte Diario Ecodinamometria M-1544 - 09-09-2025.xlsx
 - C:\MedicionesInbox\excels\2025-09-08__VARGAS, SERGIO ALEXANDER__MEDICIONES 08-09-2025\Parte Diario Ecodinamometria M-1543 - 08-09-2025.xls
 - C:\MedicionesInbox\excels\2025-09-08__VARGAS, SERGIO ALEXANDER__MEDICIONES 08-09-2025\Parte Diario Ecodinamometria M-1544 - 29-08-2025.xlsx
 - C:\MedicionesInbox\excels\2025-09-